# Welcome to Tractable's ML Classifier Workshop!

[Tractable linkedIn](https://www.linkedin.com/company/tractable/) | [Anifah](https://www.linkedin.com/in/anifah-bhadmus-891a7a116/) | [Martha](https://www.linkedin.com/in/martha-rose-robinson-14567a40/) | [Wei Ann](https://www.linkedin.com/in/weiann-heng/)

Today, we will use industry-standard techniques to build a Machine Learning (ML) Classifier using supervised learning. 

We will use our classifier to classify vehicles by type (car, truck, SUV, van). Supervised learning means that we have test data that we can compare our results to. 

Start with: A dataset that gives us images (input) and labels (teaching signal/feedback). 
End with: An array of 4 numbers for each of those images (i.e. their classification confidences!)

1. Create training, validation, and test data sets
2. Build a data pipeline
3. Explore a pre-trained model
4. Perform feature extraction to get classifier!

# Before we start
1. Housekeeping 
2. How to use Jupyter notebook
3. Quick concepts

### Neural network
Source: IBM

<img src="https://1.cms.s81c.com/sites/default/files/2021-01-06/ICLH_Diagram_Batch_01_03-DeepNeuralNetwork-WHITEBG.png" alt="neural-network" width="500px">

### Feature 
An individual measureable property/characteristic. These are usually numeric, but we can think of some qualitative examples

[Source](https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.nosimpler.me%2Fmachine-learning%2F&psig=AOvVaw1i03lInhxU-kNY9Y2zn59a&ust=1619104334232000&source=images&cd=vfe&ved=0CAoQjRxqFwoTCJiZg-jPj_ACFQAAAAAdAAAAABAJ)
<img src="https://www.nosimpler.me/wp-content/uploads/2016/08/ml-features-1.jpg" alt="feature" width="400px">

### Classifier
A classifier is the thing that we train to classify data into classes.

# Let's get started!

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
from workshop_utils import extract_data, get_data

ModuleNotFoundError: No module named 'tensorflow'

## 1. Create our datasets

### Goal: Create a training, validation, and test dataset from available data

#### Input: 
Stanford Cars196 dataset (Dataset citation: 3D Object Representations for Fine-Grained Categorization. Jonathan Krause, Michael Stark, Jia Deng, Li Fei-Fei. 4th IEEE Workshop on 3D Representation and Recognition, at ICCV 2013 (3dRR-13). Sydney, Australia. Dec. 8, 2013.)

TODO: more information about input: files and labels (0,1,2,3 which correspond to car, truck, SUV, van)


#### Output:
3 datasets (training, validation, test) that generate batches of images and class labels

In [ ]:
# Extract compressed dataseet
extract_data()

# Function to convert files into an matrix 2D images and labels.
# Each 2D image is made out of 3 channels 
def load_data(path, label):
    image = tf.image.decode_jpeg(tf.io.read_file(path), channels=3)
    return image, label

train_dataset = tf.data.Dataset.from_tensor_slices(get_data('train'))
train_dataset = train_dataset.map(load_data)

In [ ]:
# Let's take a peep at this dataset using our load_data function
fig, axarr = plt.subplots(2, 5, figsize=(10, 5))
axes = axarr.flatten()
for i, (image, label) in enumerate(train_dataset.take(10)):
    ax = axes[i]
    ax.imshow(image.numpy())
    ax.set_title(int(label))
    ax.axis("off")
plt.show()

In [ ]:
# Get 3 datasets
train_dataset = tf.data.Dataset.from_tensor_slices(get_data('train')).map(load_data)
validation_dataset = tf.data.Dataset.from_tensor_slices(get_data('val')).map(load_data)
test_dataset = tf.data.Dataset.from_tensor_slices(get_data('test')).map(load_data)


# Batch size = number of training exmaples utilized in one iteration. Important to not over/underfit
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().batch(batch_size).prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().batch(batch_size).prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.cache().batch(batch_size).prefetch(buffer_size=AUTOTUNE)

image_batch, label_batch = next(iter(train_dataset))
print(image_batch.shape)
print(label_batch.shape)

## 2. Build a data pipeline

### Goal: Create augmented data set to improve training

#### Input: 
Training dataset from previous section

#### Output:
Larger training data set with augmentation?? (this isn't actually true - we are just showing what the augumented data looks like, discuss with team)

In [ ]:
# We will be using tensorflow library
from tf.keras.layers.experimental import preprocessing

# Link to library https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing
# CODE-ALONG CHECKPOINT 1 - use tf.keras to pre-process images
data_augmentation = tf.keras.Sequential([
  preprocessing.?,
  preprocessing.?,
])

In [ ]:
for images, labels in train_dataset.take(1):
    plt.figure(figsize=(10, 10))
    first_image = images[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(
            tf.expand_dims(first_image, 0), training=True
        )
        plt.imshow(augmented_image[0].numpy().astype("int32"))
        plt.title(int(labels[0]))
        plt.axis("off")

## 3. How does a pre-trained model perform?

### Goal: Test how good Google's pre-trained MobileNet V2 model is at classifying vehicles out of the box

This is an example of how real world AI applications are built - use pre-trained model rather than building from scratch. Won't go into too much detail

#### Input: 
1. Google's MobileNet V2 model, which is pre-trained on ImageNet dataset with a wide variety of categories (not just vehicles, but things like food, people, etc)
2. Our training Cars196 dataset

#### Output:

The images of each vehicle and what MobileNet thinks this image is. (Remember that this model is general, so will not output 0,1,2,3 like you might expect.)

In [2]:
IMG_SHAPE = (160, 160, 3)

# Prepare data to suit MobileNet's expected input
# Weights trained on imagenet
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
mobilenet_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, weights='imagenet')

# Create a tf.Keras model based on our inputs and MobileNet
input_layer = tf.keras.Input(shape=IMG_SHAPE)
# We don't use this next line anywhere else, will check with team and remove
preprocessed_input_layer = preprocess_input(input_layer)
mobilenet_layer = mobilenet_model(preprocessed_inputs)
model = tf.keras.Model(input_layer, mobilenet_layer)

# Get model's predictions on our previously created image batch
prediction_batch = model(image_batch)

# Decode these predictions into human-readable class names (from imagenet)
decoded_predictions = tf.keras.applications.mobilenet_v2.decode_predictions(prediction_batch.numpy(), top=1)

fig, axarr = plt.subplots(8, 4, figsize=(10,20))
axes = axarr.flatten()
for i, (image, prediction) in enumerate(zip(image_batch, decoded_predictions)):
    ax = axes[i]
    ax.imshow(image.numpy())
    top_label = prediction[0][1]
    score = prediction[0][2]
    ax.set_title(f'{top_label}, {score:.2f}')
    ax.axis('off')
plt.tight_layout()
plt.show()

NameError: name 'tf' is not defined

## 4. Feature extraction

### Goal: Use an existing model to extract features from our images, and transfer that knowledge to our new task

*Use mobilenet v2 to extract useful visual features and build a new classifier on top*

In [1]:
# TODO: 
# Diagram of what we're trying to do. Feature extractor and what we're trying to get out, classifier head and how that accepts features.
# What bits we will train.
# Step through entire plan before we write any code.

### Step by step runthrough:
1. Create a base model from the pre-trained model MobileNet V2 and change it's output from classifications to features
2. Create a "feature extractor" that converts a visual image into a block of features
3. Generate predictions from these blocks of features. (need more explanation)

#### Step 1: Create the base model from the pre-trained model MobileNet V2 and change it's output from classifications to features

This feature extractor converts each 160x160x3 image into a 5x5x1280 block of features. Let's see what it does to an example batch of images:


In [ ]:
IMG_SHAPE = (160, 160, 3)
# CODE-ALONG CHECKPOINT 2
base_model = ?
# make it untrainable
?

In [ ]:
# Let's take a look at the base model architecture
base_model.summary()

#### STEP 2 - Create a "feature extractor" that converts a visual image into a block of features

Feature extractor converts 160x160x3 image into a 5x5x1280 block of features.

In [2]:
image_batch, label_batch = next(iter(train_dataset))

# CODE-ALONG CHECKPOINT 2
feature_batch = ?
print(?)

NameError: name 'train_dataset' is not defined

In [ ]:
#### STEP 3 - Generate predictions from these blocks of features.

In [ ]:
Now, we need to generate predictions from these blocks of features. 
First, create a layer to average over the 5x5 spatial locations, creating a single 1280 vector per image:

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

Next, create a fully-connected layer of neurons as our outputs for our 4 classes. Each output neuron should be connected to each of the 1280 neurons in the previous layer.

In [ ]:
# CODE-ALONG CHECKPOINT 4 - Create a fully connected output layer

Now let's connect all our parts together to build our model.

In [ ]:
# CODE-ALONG CHECKPOINT 5 - Define our model


Compile the model before training it. We need to pick a learning rate, an optimiser, a loss function, and any metrics we want to track.

In [ ]:
# Compile our model

Actually train the model!!

In [ ]:
# Evaluate prior to training

In [ ]:
# Train

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
#Retrieve a batch of images from the test set
image_batch, label_batch = test_dataset.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch)

In [ ]:
class_names = ['Car', 'SUV', 'Truck', 'Van']
fig, axarr = plt.subplots(8, 4, figsize=(10,20))
axes = axarr.flatten()
for i, (image, prediction) in enumerate(zip(image_batch, predictions)):
    ax = axes[i]
    ax.imshow(image)
    top_label = np.argmax(prediction)
    ax.set_title(f'{class_names[top_label]}')
    ax.axis('off')
plt.tight_layout()
plt.show()

# Fine tuning!

This is your challenge.

In feature extraction, we were training a small number of new layers on top of the existing MobileNetV2 base model. The weights of that base model were not updated during training.

One way to increase performance further is to "fine-tune" the model, by training the weights of the top layers of the base model alongside our classification layer. This processes should force the weights of the base model to move from defining generic feature maps to features more specific for our task.

(Note, if you try this without first training the top-level classifier with the base model set to non-trainable, the graident updates will be very large (because the classifier starts off with random weights) and your base model is likely to forget what it has already learned).

There are quite a few hyperparameters to change here -- how many layers to allow fine-tuning for, what learning rate to use, etc. We've set you up with a starting point, but play around and see how much you can improve performance with fine-tuning. We will have a prize for the best improvement!

Un-freeze the top layers of the model
All you need to do is unfreeze the base_model and set the bottom layers to be un-trainable. Then, you should recompile the model (necessary for these changes to take effect), and resume training.

In [ ]:
base_model.trainable = True

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

Compile the model

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])

model.summary()
len(model.trainable_variables)


Continue training the model - if trained to convergence earlier this will improve your model

In [ ]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_dataset)

Results of fine tuning

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

Evaluation and prediction

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

In [ ]:
#Retrieve a batch of images from the test set
image_batch, label_batch = test_dataset.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch)

In [ ]:
class_names = ['Car', 'SUV', 'Truck', 'Van']
fig, axarr = plt.subplots(8, 4, figsize=(10,20))
axes = axarr.flatten()
for i, (image, prediction) in enumerate(zip(image_batch, predictions)):
    ax = axes[i]
    ax.imshow(image)
    top_label = np.argmax(prediction)
    score = prediction[top_label]
    ax.set_title(f'{class_names[top_label]}, {score:.2f}')
    ax.axis('off')
plt.tight_layout()
plt.show()